<a href="https://colab.research.google.com/github/adityasaich/sasi_btp/blob/main/sasi_btp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id = '18xGNPuvbXGSN97g0CzI1tIBl81zJrToF'

In [4]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dataset.csv')  
df_input = pd.read_csv('dataset.csv')
#df_input = df_input.sample(n=50000)

In [ ]:
#url = 'https://raw.githubusercontent.com/adityasaich/sasi_btp/main/dataworld_set.csv?token=ALPKPQCX3COGEIP65QDO2V3BR5QT6'
#df_input = pd.read_csv(url)

In [5]:
def deviationTransform(arr):
  m = np.mean(arr)
  d = np.std(arr)
  return [m,d]
def minMaxTransform(arr):
  min = np.min(arr)
  max = np.max(arr)
  return [min,max-min]

In [6]:
#pre-processing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
df_input = df_input[df_input['Area'] > 0 ]
df_input = df_input[df_input['Production'] > 0 ]
df_input["ProductionPerArea"] = ((df_input["Production"])/(df_input["Area"]))
#dropping columns which are not used
df_input = df_input.drop(columns=['District_Name','Area','Production'])
#replace empty strings with nan
df_input = df_input.replace(r'^\s*$', np.NaN, regex=True)
#drop null values
df_input = df_input.dropna()


In [ ]:
x = df_input['ProductionPerArea']
print(x.describe())

import matplotlib.pyplot as plt
plt.style.use('ggplot')
fig, ax = plt.subplots()
ax.boxplot((x), vert=False, showmeans=True, meanline=True,
           labels=('x'), patch_artist=True,
           medianprops={'linewidth': 2, 'color': 'purple'},
           meanprops={'linewidth': 2, 'color': 'red'})
plt.show()
hist, bin_edges = np.histogram(x, bins=20)
fig, ax = plt.subplots()
ax.hist(x, bin_edges, cumulative=False)
ax.set_xlabel('x')
ax.set_ylabel('Frequency')
plt.show()

In [7]:
categorical_columns = ['State_Name', 'Crop' ,'Season']
#label encoder dict
labels_dict = {}
#scaling dict
scaling_dict = {}
for column in categorical_columns:
  le = LabelEncoder()
  le.fit(df_input[column])
  df_input[column] = le.transform(df_input[column])
  labels_dict[column] = list(le.classes_)
  scaling_params = minMaxTransform(np.array(df_input[column]))
  df_input[column] = (df_input[column] - scaling_params[0])/scaling_params[1]
  scaling_dict[column] = scaling_params
scaling_params = deviationTransform(np.array(df_input['Crop_Year'])) 
scaling_dict['Crop_Year'] = scaling_params
df_input['Crop_Year'] = (df_input['Crop_Year'] - scaling_params[0])/scaling_params[1]
scaling_params = deviationTransform(np.array(df_input['ProductionPerArea']))
df_input['ProductionPerArea'] = (df_input['ProductionPerArea'] - scaling_params[0])/scaling_params[1]
scaling_dict['ProductionPerArea'] = scaling_params

In [8]:
df_input

,State_Name,Crop_Year,Season,Crop,ProductionPerArea
0,0.0,-1.140834,0.2,0.009615,-0.049381
1,0.0,-1.140834,0.2,0.625000,-0.050711
2,0.0,-1.140834,0.2,0.769231,-0.047496
3,0.0,-1.140834,0.8,0.048077,-0.046895
4,0.0,-1.140834,0.8,0.173077,-0.051039
...,...,...,...,...,...
246086,1.0,1.672975,0.6,0.769231,-0.048139
246087,1.0,1.672975,0.6,0.836538,-0.050421
246088,1.0,1.672975,0.8,0.865385,0.009581
246089,1.0,1.672975,1.0,0.769231,-0.048717


In [9]:
from sklearn.model_selection import train_test_split
df_small = df_input
df_small.columns.name = None
df=df_small
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.33, random_state=42)

In [10]:
from sklearn.metrics import mean_squared_error
def classify(clf,x_train,x_test,y_train,y_test):
  clf.fit(x_train,y_train)
  y_pred = clf.predict(x_test)
  print(y_pred)
  y_train_pred = clf.predict(x_train)
  print(mean_squared_error(y_train, y_train_pred))
  print(mean_squared_error(y_test, y_pred))
  y_pred = np.array(y_pred)
  y_test = np.array(y_test)
  for i in range(y_test.size):
    if(y_test[i]>15):
      print(i,y_test[i],y_pred[i])

In [11]:
from sklearn.ensemble import RandomForestRegressor
regresser = RandomForestRegressor(n_estimators = 10 ,random_state = 0)
print("\t\t\t random-forest classifier")
classify(regresser,x_train,x_test,y_train,y_test)

			 random-forest classifier
[ 0.01718748 -0.05048732 -0.04854881 ... -0.05052442 -0.02678204
 -0.04799629]
0.07646626095892917
0.08404042773276447
1016 18.048157387148553 10.790696503551965
2301 19.178733980603386 16.569626947481094
3379 15.733750015519044 17.8666979937099
4548 15.579384195135523 13.780943667245884
6874 15.47750351558056 14.192251040871133
7157 20.0614115921627 14.192251040871133
7629 31.847673359127906 7.916022847728522
8261 15.580982798702802 14.65579092090132
8562 16.657244942664597 16.569626947481094
13028 16.952992858209107 14.085195033722508
14294 16.826117132638228 11.249560264553674
15834 21.914204130639654 14.192251040871133
16764 22.812465342203193 11.26670951031566
18342 24.3111404078194 16.97968970598324
19189 16.657625219165993 11.249560264553674
19923 16.88143553172169 14.085195033722508
21507 17.630229481228305 16.569626947481094
21747 15.447274426707304 10.83136501420579
21865 40.126543306457734 19.038918852125423
22908 16.02064104619277 7.916022847728

In [ ]:
regresser.get_params("true")

In [ ]:
#linear regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
print("\t\t\t linear-regression classifier")
classify(reg,x_train,x_test,y_train,y_test)

			 linear-regression classifier
[ 0.05215216  0.01925075  0.056664   ... -0.11000213 -0.09175651
  0.05040157]
1.1102781920460332
0.7541618347330906
1016 18.048157387148553 0.1894157364236274
2301 19.178733980603386 0.16451249230592974
3379 15.733750015519044 0.14278598796793596
4548 15.579384195135523 0.17905418374970355
6874 15.47750351558056 0.20128169722701184
7157 20.0614115921627 0.20128169722701184
7629 31.847673359127906 0.16083750480994535
8261 15.580982798702802 0.14596134703612285
8562 16.657244942664597 0.16451249230592974
13028 16.952992858209107 0.194930979090638
14294 16.826117132638228 0.17086321044230351
15834 21.914204130639654 0.20128169722701184
16764 22.812465342203193 0.16284154136720336
18342 24.3111404078194 0.1981063381588249
19189 16.657625219165993 0.17086321044230351
19923 16.88143553172169 0.194930979090638
21507 17.630229481228305 0.16451249230592974
21747 15.447274426707304 0.18624037735544052
21865 40.126543306457734 0.20445705629519872
22908 16.0206410

In [12]:
json_params = {}
json_params['labels'] = labels_dict
json_params['scaling'] = scaling_dict

In [13]:
json_params

{'labels': {'Crop': ['Arcanut (Processed)',
   'Arecanut',
   'Arhar/Tur',
   'Atcanut (Raw)',
   'Bajra',
   'Banana',
   'Barley',
   'Bean',
   'Beans & Mutter(Vegetable)',
   'Bhindi',
   'Bitter Gourd',
   'Black pepper',
   'Blackgram',
   'Bottle Gourd',
   'Brinjal',
   'Cabbage',
   'Cardamom',
   'Carrot',
   'Cashewnut',
   'Cashewnut Processed',
   'Cashewnut Raw',
   'Castor seed',
   'Cauliflower',
   'Citrus Fruit',
   'Coconut ',
   'Coffee',
   'Colocosia',
   'Cond-spcs other',
   'Coriander',
   'Cotton(lint)',
   'Cowpea(Lobia)',
   'Drum Stick',
   'Dry chillies',
   'Dry ginger',
   'Garlic',
   'Ginger',
   'Gram',
   'Grapes',
   'Groundnut',
   'Guar seed',
   'Horse-gram',
   'Jack Fruit',
   'Jobster',
   'Jowar',
   'Jute',
   'Jute & mesta',
   'Kapas',
   'Khesari',
   'Korra',
   'Lemon',
   'Lentil',
   'Linseed',
   'Maize',
   'Mango',
   'Masoor',
   'Mesta',
   'Moong(Green Gram)',
   'Moth',
   'Niger seed',
   'Oilseeds total',
   'Onion',
   'Oran